In [ ]:
from google.colab import drive
from random import randint
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

from typing import Tuple, List

from two_player_games.games.connect_four import ConnectFour, ConnectFourMove
from two_player_games.player import Player
from two_player_games.state import State
import gc
import time
from matplotlib import pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [ ]:
class MinMaxSolver:
    def __init__(self, game: ConnectFour):
        self.game = game
        self.max_player = self.game.first_player
        self.min_player = self.game.second_player

    def swap_players(self):
        self.max_player, self.min_player = self.min_player, self.max_player

    def evaluate_position(self)->float:
        fields = self.game.state.fields
        score = 0

        for column in range(len(fields)): # Check every column
            for row in range(len(fields[column]) - 3):
                fragment = fields[column][row:row+4]
                score += self.evaluate_fragment(fragment)

        for column in range(len(fields) - 3): # Check every row
            for row in range(len(fields[column])):
                fragment = [fields[column+i][row] for i in range(4)]
                score += self.evaluate_fragment(fragment)

        for column in range(len(fields) - 3): # Check every left-down to right-up diagonal
            for row in range(len(fields[column]) - 3):
                fragment = [fields[column+i][row+i] for i in range(4)]
                score += self.evaluate_fragment(fragment)

        for column in range(len(fields) - 3): # Check every left_up to right-down diagonal
            for row in range(3, len(fields[column])):
                fragment = [fields[column+i][row-i] for i in range(4)]
                score += self.evaluate_fragment(fragment)

        return score

    def get_best_move(self, depth:int)->int:
        return self.minimax(depth, alpha=float('-inf'), beta=float('inf'), is_maximizing_player=True)[0]

    def get_best_move_no_pruning(self, depth:int)->int:
        return self.minimax_no_pruning(depth, alpha=float('-inf'), beta=float('inf'), is_maximizing_player=True)[0]

    def is_valid_move(self, col_index:int)->bool:
        possible_moves = game.get_moves()
        possible_indexes = [possible_moves[i].column for i in range(len(possible_moves))]
        return col_index in possible_indexes

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        if depth == 0 or game.is_finished():
            return -1, self.evaluate_position()
        best_move = -1

        if is_maximizing_player:
            for move in game.get_moves():
                old_state = self.game.state
                self.game.make_move(move)
                eval = self.minimax(depth-1, alpha, beta, False)[1]
                self.game.state = old_state
                if eval > alpha:
                    alpha = eval
                    best_move = move.column
                if beta <= alpha:
                    break
            return best_move, alpha
        else:
            for move in game.get_moves():
                old_state = self.game.state
                self.game.make_move(move)
                eval = self.minimax(depth-1, alpha, beta, True)[1]
                self.game.state = old_state
                if eval < beta:
                    beta = eval
                    best_move = move.column
                if beta <= alpha:
                    break
            return best_move, beta

    def minimax_no_pruning(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        if depth == 0 or game.is_finished():
            return -1, self.evaluate_position()
        best_move = -1

        if is_maximizing_player:
            for move in game.get_moves():
                old_state = self.game.state
                self.game.make_move(move)
                eval = self.minimax(depth-1, alpha, beta, False)[1]
                self.game.state = old_state
                if eval > alpha:
                    alpha = eval
                    best_move = move.column
            return best_move, alpha
        else:
            for move in game.get_moves():
                old_state = self.game.state
                self.game.make_move(move)
                eval = self.minimax(depth-1, alpha, beta, True)[1]
                self.game.state = old_state
                if eval < beta:
                    beta = eval
                    best_move = move.column
            return best_move, beta

    def evaluate_fragment(self, fragment: Tuple[int])->float:
        score = 0

        if fragment.count(self.max_player) == 4:
            score += 1000000
        elif fragment.count(self.max_player) == 3 and fragment.count(None) == 1:
            score += 1000
        elif fragment.count(self.max_player) == 2 and fragment.count(None) == 2:
            score += 1

        if fragment.count(self.min_player) == 4:
                score -= 1000000
        elif fragment.count(self.min_player) == 3 and fragment.count(None) == 1:
                score -= 1000
        elif fragment.count(self.min_player) == 2 and fragment.count(None) == 2:
            score -= 1

        return score

In [ ]:
p1 = Player("a")
p2 = Player("b")

for depth in range(1, 8):
  game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
  solver = MinMaxSolver(game)
  print("Depth: ", depth)
  while not game.is_finished():
      game.make_move(ConnectFourMove(solver.get_best_move(depth)))
      solver.swap_players()
      print(game, '\n')
  print("Winner", game.get_winner().char) if game.get_winner() else print("Draw")
  print(60*'=', '\n')

Depth:  1
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ] 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ] 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][a][ ][ ][ ][ ][ ] 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][a][ ][ ][ ][ ][ ] 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][a][a][ ][ ][ ][ ] 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][a][a][b][ ][ ][ ] 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

Niezależnie od głębokości solvera, rozgrywki pokrywają praktycznie całą część planszy

In [ ]:
print("We test the result for 20 games\n")
for depth in range(1, 9):
  winners = []
  for i in range(20):
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
    solver = MinMaxSolver(game)

    game.make_move(ConnectFourMove(randint(0, COLUMN_COUNT - 1)))
    game.make_move(ConnectFourMove(randint(0, COLUMN_COUNT - 1)))

    while not game.is_finished():
        game.make_move(ConnectFourMove(solver.get_best_move(depth)))
        solver.swap_players()
    winners.append(game.get_winner())
  print("Depth: ", depth)
  print("Player 1 won: ", winners.count(p1))
  print("Player 2 won: ", winners.count(p2))
  print("Draws: ", winners.count(None), '\n')

We test the result for 20 games

Depth:  1
Player 1 won:  8
Player 2 won:  12
Draws:  0 

Depth:  2
Player 1 won:  7
Player 2 won:  13
Draws:  0 

Depth:  3
Player 1 won:  12
Player 2 won:  8
Draws:  0 

Depth:  4
Player 1 won:  6
Player 2 won:  10
Draws:  4 

Depth:  5
Player 1 won:  11
Player 2 won:  9
Draws:  0 

Depth:  6
Player 1 won:  7
Player 2 won:  11
Draws:  2 

Depth:  7
Player 1 won:  10
Player 2 won:  9
Draws:  1 

Depth:  8
Player 1 won:  8
Player 2 won:  7
Draws:  5 



In [ ]:
for depth in range(1, 4):
  winners = []
  for i in range(50):
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
    solver = MinMaxSolver(game)

    game.make_move(ConnectFourMove(randint(0, COLUMN_COUNT - 1)))
    game.make_move(ConnectFourMove(randint(0, COLUMN_COUNT - 1)))

    while not game.is_finished():
        game.make_move(ConnectFourMove(solver.get_best_move(depth)))
        solver.swap_players()
    winners.append(game.get_winner())
  print("Depth: ", depth)
  print("Player 1 won: ", winners.count(p1))
  print("Player 2 won: ", winners.count(p2))
  print("Draws: ", winners.count(None), '\n')

Depth:  1
Player 1 won:  25
Player 2 won:  25
Draws:  0 

Depth:  2
Player 1 won:  22
Player 2 won:  27
Draws:  1 

Depth:  3
Player 1 won:  33
Player 2 won:  17
Draws:  0 



Przy mniejszych wartościach głębokości przeszukiwania rozgrywki bardzo rzadko kończą się remisem. Wraz ze wzrostem głębokości wzrasta ilość remisów jako wyniki.

Nie dostrzegam większej zależności pomiędzy głębokością przeszukiwania a tym, który z graczy zwycięży. Wyniki uzyskane są obarczone pewną niepewnością wynikającą z tego, że pierwsze dwa posunięcia wybierane są losowo. Natomiast można zauważyć, że żaden z graczy nie uzykuje znaczącej przewagi z kolejności, w której wykonuje ruchy.